# WhisperTS joining JSON to SRT

In [26]:
import re,codecs,json,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

In [30]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/JSON/"
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/Segmenter/Segments/JSON/"

In [31]:
json1=repJson+"extrait-00.wav.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [32]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [33]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [34]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [35]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [36]:
def findSample(sample,ref):
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if sample==ref[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [37]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        return None

In [38]:
for i in range(1,14):
    print(i)
    json2=repJson+"extrait-%02d.wav.json"%i
    with open(json2) as inFile:
        whisper2=json.load(inFile)
    newSrt=pasteWords(srt,whisper2)
    if not newSrt:
        break
    else:
        srt=newSrt
# print(srt)

1
8
found 1 92
found 1 92 tout 00:01:00,439 tout 00:00:00,440
00:00:59,999
('00:01:00,619', '00:01:00,859', 'un', 0.863) ('00:01:00,618', '00:01:00,879', 'un', 0.772)
2
8
found 1 261
found 1 261 il 00:02:05,457 il 00:00:04,320
00:02:00,498
('00:02:05,539', '00:02:05,598', 'y', 0.999) ('00:02:05,539', '00:02:06,039', 'y', 0.765)
3
8
found 22 461
found 22 461 au 00:03:20,399 au 00:00:19,820
00:03:00,497
('00:03:20,498', '00:03:20,759', 'chef', 0.958) ('00:03:20,497', '00:03:20,798', 'chef', 0.999)
4
8
found 49 627
found 49 627 il 00:04:22,837 il 00:00:19,760
00:04:03,437
('00:04:23,257', '00:04:23,697', 'y', 1.0) ('00:04:23,257', '00:04:23,357', 'y', 1.0)
5
8
found 9 724
found 9 724 quils 00:05:02,716 quils 00:00:02,359
00:05:00,656
('00:05:03,257', '00:05:04,216', 'construisent', 0.636) ('00:05:03,257', '00:05:03,915', 'construisent', 0.993)
6
8
found 1 878
found 1 878 y 00:06:00,476 y 00:00:00,500
00:06:00,056
('00:06:00,576', '00:06:00,757', 'a', 0.991) ('00:06:00,576', '00:06:00,596'

In [39]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [40]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:09,339 C'est à cet âge, entre 2 et 6 ans, [*] que se joue [*] la personnalité [*] future de l'enfant, [*] son équilibre, [*] son harmonie.
00:00:14,140 00:00:53,478 Aux portes de l'école maternelle, ils étaient 1 287 000 [*] à la dernière rentrée. Aux [*] portes de l'école maternelle, [*] ils étaient 1 [*] 287 000 à la dernière rentrée. C'est un peu comme se décharger de son enfant, comme on se décharge d'un fardeau.
00:00:53,859 00:00:54,539 Pour d'autres...
00:00:56,119 00:00:59,439 C'est excellent [*] pour les parents qui ne savent pas élever leurs enfants en bas âge.
00:01:00,259 00:01:05,298 C'est un [*] savoir de pouvoir enseigner quelque chose aux enfants, je pense.
00:01:06,198 00:01:08,518 J'attends une préparation pour l'école primaire.
00:01:09,198 00:01:14,638 Et puis [*] ça les met en plus en contact avec les autres, ça va être une manière plus [*] sociale, je crois que c'est bien.
00:01:14,938 00:01:19,799 Ils aiment l'école, ils ont envie d'y venir, il

In [41]:
with codecs.open(json1.replace(".json","-turns.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")